In [1]:
import json
import pandas as pd

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/stage2_1989_57/without_head/conversations.json'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/stage2_1989_57/without_head/graph.jsonl'

In [3]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

# Example usage
input_text = """
some initial text
module name="example1" \n   attribu\nte="   value";
some other text
module name="example2" attribute="another value";
"""

# Find the first occurrence
result = find_module_string(input_text)
print("Found substring:", result)

# If you need to find all occurrences instead of just the first one, you can use re.findall
all_matches = re.findall(r"module.*?;", input_text, re.DOTALL)
print("All found substrings:", all_matches)

Found substring: module name="example1" 
   attribu
te="   value";
All found substrings: ['module name="example1" \n   attribu\nte="   value";', 'module name="example2" attribute="another value";']


In [4]:
graph = pd.read_json(graph_path, lines=True)
with open(conv_path, 'r') as f:
    conv = json.load(f)


In [5]:
conv[0]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'a', 'type': 'input port'}, {'id': 3, 'content': 'b', 'type': 'input port'}, {'id': 4, 'content': 'opcode', 'type': 'input port'}, {'id': 5, 'content': 'result', 'type': 'output port'}, {'id': 6, 'content': 'error', 'type': 'output port'}, {'id': 7, 'content': 'u_adder', 'type': 'submodule'}, {'id': 8, 'content': 'u_subtractor', 'type': 'submodule'}, {'id': 9, 'content': 'u_and', 'type': 'submodule'}, {'id': 10, 'content': 'u_or', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[2, 3, 2, 3, 2, 3, 2, 3, 7, 9, 8, 10], [7, 7, 8, 8, 9, 9, 10, 10, 5, 5, 5, 5]]\n, Implement a modular Arithmetic Logic Unit (ALU) for 32-bit integers in Verilog. The top-level alu module should instantiate submodules for addition, subtraction, bitwise AND, and bitwise OR operations. The alu module uses these submodul

In [6]:
conv[0]['conversations'][1]['value']

"module alu(\n    input clk,\n    input rst,\n    input [31:0] a,\n    input [31:0] b,\n    input [3:0] opcode,\n    output reg [31:0] result,\n    output reg error\n);\n\nwire [31:0] sum, difference, and_result, or_result;\n\n// Instantiating the adder module\nadder u_adder (\n    .a(a),\n    .b(b),\n    .sum(sum)\n);\n\n// Instantiating the subtractor module\nsubtractor u_subtractor (\n    .a(a),\n    .b(b),\n    .difference(difference)\n);\n\n// Instantiating the and module\nand_module u_and (\n    .a(a),\n    .b(b),\n    .and_result(and_result)\n);\n\n// Instantiating the or module\nor_module u_or (\n    .a(a),\n    .b(b),\n    .or_result(or_result)\n);\n\nalways @(posedge clk or posedge rst) begin\n    if (rst) begin\n        result <= 0;\n        error <= 0;\n    end else begin\n        case(opcode)\n            4'b0000: result <= sum;\n            4'b0001: result <= difference;\n            4'b0010: result <= and_result;\n            4'b0011: result <= or_result;\n            de

In [7]:
head = find_module_string(conv[0]['conversations'][1]['value'])
conv[0]['conversations'][0]['value'] + head

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'a', 'type': 'input port'}, {'id': 3, 'content': 'b', 'type': 'input port'}, {'id': 4, 'content': 'opcode', 'type': 'input port'}, {'id': 5, 'content': 'result', 'type': 'output port'}, {'id': 6, 'content': 'error', 'type': 'output port'}, {'id': 7, 'content': 'u_adder', 'type': 'submodule'}, {'id': 8, 'content': 'u_subtractor', 'type': 'submodule'}, {'id': 9, 'content': 'u_and', 'type': 'submodule'}, {'id': 10, 'content': 'u_or', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[2, 3, 2, 3, 2, 3, 2, 3, 7, 9, 8, 10], [7, 7, 8, 8, 9, 9, 10, 10, 5, 5, 5, 5]]\n, Implement a modular Arithmetic Logic Unit (ALU) for 32-bit integers in Verilog. The top-level alu module should instantiate submodules for addition, subtraction, bitwise AND, and bitwise OR operations. The alu module uses these submodul

In [8]:
for i in range(len(conv)):
    module_head = find_module_string(conv[i]['conversations'][1]['value'])
    conv[i]['conversations'][0]['value'] = conv[i]['conversations'][0]['value'] + '\n' + module_head

In [9]:
# conv[100]['conversations'][0]['value']

In [10]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/stage2_1989_57/with_head/conversations.json'



In [11]:
with open(new_conv_path, 'w') as f:
    json.dump(conv, f)

In [12]:
import json
with open(new_conv_path, 'r') as f:
    new_conv = json.load(f)
    


In [13]:
len(new_conv)

480

In [ ]:
new_conv[100]['conversations'][0]['value']

In [32]:
len(graph)

2148